In [1]:
import pandas as pd
import openpyxl
import numpy as np
import plotly.express as px

In [2]:
base = pd.read_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/base_analisis.xlsx")
base['Fecha'] = pd.to_datetime(base['Fecha'])
base.head(5)

,Fecha,Producto,Posicion,Empresa,Mes,Año,coeficiente,prima_compradora,prima_vendedora,Precio,...,st_vendedor,Categoria,omitido_comprador,omitido_vendedor,posicion,time_delta_months,Embarque,Valor,Año_Fecha,Mes_Fecha
0,2020-01-02,Aceite de Girasol,NaN,JJH,1,2020,1.0,775.0,810.0,0.0,...,11.81,broker,0.0,0.0,2020-01-01,0,t+0,Amarillo,2020,1
1,2020-01-02,Aceite de Girasol,NaN,Glencore,1,2020,1.0,750.0,810.0,0.0,...,11.81,empresa,1.0,0.0,2020-01-01,0,t+0,Amarillo,2020,1
2,2020-01-02,Aceite de Girasol,NaN,Agro Sud,1,2020,1.0,780.0,800.0,0.0,...,11.81,broker,0.0,0.0,2020-01-01,0,t+0,Amarillo,2020,1
3,2020-01-02,Aceite de Girasol,NaN,Cargill,1,2020,1.0,NaN,785.0,0.0,...,11.81,empresa,NaN,1.0,2020-01-01,0,t+0,Amarillo,2020,1
4,2020-01-02,Aceite de Girasol,NaN,JJH,2,2020,1.0,775.0,810.0,0.0,...,14.72,broker,0.0,0.0,2020-02-01,1,t+1,Verde,2020,1


In [3]:
print(base.dtypes)

Fecha                datetime64[ns]
Producto                     object
Posicion                     object
Empresa                      object
Mes                           int64
Año                           int64
coeficiente                 float64
prima_compradora            float64
prima_vendedora             float64
Precio                      float64
precio_comprador            float64
precio_vendedor             float64
media_compradora            float64
media_vendedor              float64
st_comprador                float64
st_vendedor                 float64
Categoria                    object
omitido_comprador           float64
omitido_vendedor            float64
posicion             datetime64[ns]
time_delta_months             int64
Embarque                     object
Valor                        object
Año_Fecha                     int64
Mes_Fecha                     int64
dtype: object


PRECIOS - EMPRESA

In [4]:
empresa = base[base['Categoria'] == 'empresa']
#Calculo los promedios 
resumen = empresa.groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_comprador'].mean().reset_index()
resumen_vendedor = empresa.groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_vendedor'].mean().reset_index()

#Junto los promedios y redondeo a 2 decimales
resumen_empresa = resumen.merge(resumen_vendedor, on=['Fecha', 'Producto', 'Año', 'Mes'], how='left')
resumen_empresa['precio_comprador'] = resumen_empresa['precio_comprador'].round(2)
resumen_empresa['precio_vendedor'] = resumen_empresa['precio_vendedor'].round(2)

# Calculo el precio promedio de la empresa
resumen_empresa['precio_empresa'] = round((resumen_empresa['precio_comprador'] + resumen_empresa['precio_vendedor']) / 2, 2)

# Elimino variables 
del resumen
del resumen_vendedor

In [ ]:
resumen_empresa = resumen_empresa[['Fecha', 'Producto', 'Mes', 'Año', 'precio_empresa']]
print(resumen_empresa.dtypes)

In [6]:
resumen_empresa["posicion"] = pd.to_datetime(resumen_empresa["Año"].astype(str) + '-' + resumen_empresa["Mes"].astype(str) + '-01')
resumen_empresa['posicion'] = pd.to_datetime(resumen_empresa['posicion'])

resumen_empresa['time_delta_months'] = resumen_empresa['posicion'].dt.to_period('M').view(dtype='int64') - resumen_empresa['Fecha'].dt.to_period('M').view(dtype='int64')

resumen_empresa['Embarque'] ='t+' + resumen_empresa['time_delta_months'].astype(str)

PRECIOS - BROKER

In [7]:
broker = base[base['Categoria'] == 'broker']
#Calculo los promedios 
resumen = broker.groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_comprador'].mean().reset_index()
resumen_vendedor = broker.groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_vendedor'].mean().reset_index()

#Junto los promedios y redondeo a 2 decimales
resumen_broker = resumen.merge(resumen_vendedor, on=['Fecha', 'Producto', 'Año', 'Mes'], how='left')
resumen_broker['precio_comprador'] = resumen_broker['precio_comprador'].round(2)
resumen_broker['precio_vendedor'] = resumen_broker['precio_vendedor'].round(2)

# Calculo el precio promedio de la empresa
resumen_broker['precio_broker'] = round((resumen_broker['precio_comprador'] + resumen_broker['precio_vendedor']) / 2, 2)

# Elimino variables 
del resumen
del resumen_vendedor

In [8]:
resumen_broker = resumen_broker[['Fecha', 'Producto', 'Mes', 'Año', 'precio_broker']]

In [9]:
resumen_broker["posicion"] = pd.to_datetime(resumen_broker["Año"].astype(str) + '-' + resumen_broker["Mes"].astype(str) + '-01')
resumen_broker['posicion'] = pd.to_datetime(resumen_broker['posicion'])

resumen_broker['time_delta_months'] = resumen_broker['posicion'].dt.to_period('M').view(dtype='int64') - resumen_broker['Fecha'].dt.to_period('M').view(dtype='int64')

resumen_broker['Embarque'] ='t+' + resumen_broker['time_delta_months'].astype(str)

PRECIOS - INDICE

In [12]:
bolsa = pd.read_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Input/serie_fob.xlsx")
bolsa = bolsa.dropna(subset=["Precio"]).rename(columns={"Precio": "FOB_Bolsa"})
bolsa["Fecha"] = pd.to_datetime(bolsa["Fecha"], format="%d/%m/%Y")
bolsa["Cereal"].replace("Trigo 11.5%", "Trigo", inplace=True)

bolsa.rename(columns={"Cereal": "Producto"}, inplace=True)

In [13]:
bolsa["posicion"] = pd.to_datetime(bolsa["Año"].astype(str) + '-' + bolsa["Mes"].astype(str) + '-01')
bolsa['posicion'] = pd.to_datetime(bolsa['posicion'])

bolsa['time_delta_months'] = bolsa['posicion'].dt.to_period('M').view(dtype='int64') - bolsa['Fecha'].dt.to_period('M').view(dtype='int64')

bolsa['Embarque'] ='t+' + bolsa['time_delta_months'].astype(str)

TODO

In [12]:
full = bolsa.merge(resumen_broker, on=['Fecha', 'Producto', 'Mes', 'Año', 'posicion', 'time_delta_months', 'Embarque'], how='left')
full = full.merge(resumen_empresa, on=['Fecha', 'Producto', 'Mes', 'Año', 'posicion', 'time_delta_months', 'Embarque'], how='left')

In [ ]:
print(full.dtypes)

In [14]:
full = full.melt(id_vars=['Fecha', 'Producto', 'Embarque'], value_vars=[ 'FOB_Bolsa', 'precio_empresa', 'precio_broker'], var_name='Fuente', value_name='Precio')

GRAFICOS

In [ ]:
resultado = full[full['Embarque'].isin(['t+0', 't+1', 't+2', 't+3', 't+4'])]

for producto in resultado['Producto'].unique():
    resultado_producto = resultado[resultado['Producto'] == producto]
    fig2 = px.line(resultado_producto, x='Fecha', y='Precio', color='Fuente', facet_col='Embarque',
                  title='Precios ' + producto,
                  labels={'Precio': 'Precio', 'Fecha': 'Fecha', 'Fuente': 'Fuente'})
    fig2.show()
    #fig2.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/precios_" + producto + ".html")



SIN OUTLIERS


In [4]:
base_sinout = pd.read_excel("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/base_precios_sin_outliers.xlsx")
base_sinout['Fecha'] = pd.to_datetime(base_sinout['Fecha'])
base_sinout.head(5)
base_sinout["Producto"].replace("Trigo 11.5%", "Trigo", inplace=True)
print(base_sinout.dtypes)


Fecha                datetime64[ns]
Producto                     object
Posicion                     object
Empresa                      object
Mes                         float64
Año                         float64
coeficiente                 float64
prima_compradora            float64
prima_vendedora             float64
Precio                      float64
precio_comprador            float64
precio_vendedor             float64
Categoria                    object
Omitido_Comprador              bool
Omitido_Vendedor               bool
dtype: object


EMPRESA SIN OUTLIERS

In [5]:
empresa_sinout = base_sinout[base_sinout['Categoria'] == 'empresa']

#Calculo los promedios 
resumen = empresa_sinout .groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_comprador'].mean().reset_index()
resumen_vendedor = empresa_sinout .groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_vendedor'].mean().reset_index()

#Junto los promedios y redondeo a 2 decimales
resumen_empresa_sinout  = resumen.merge(resumen_vendedor, on=['Fecha', 'Producto', 'Año', 'Mes'], how='left')
resumen_empresa_sinout['precio_comprador'] = resumen_empresa_sinout['precio_comprador'].round(2)
resumen_empresa_sinout['precio_vendedor'] = resumen_empresa_sinout['precio_vendedor'].round(2)

# Calculo el precio promedio de la empresa_sinout 
resumen_empresa_sinout['precio_empresa_sinout'] = round(resumen_empresa_sinout[['precio_comprador', 'precio_vendedor']].mean(axis=1, skipna=True), 2)
#resumen_empresa_sinout['precio_empresa_sinout'] = round((resumen_empresa_sinout['precio_comprador'] + resumen_empresa_sinout['precio_vendedor']) / 2, 2)

# Elimino variables 
del resumen
del resumen_vendedor

resumen_empresa_sinout  = resumen_empresa_sinout[['Fecha', 'Producto', 'Mes', 'Año', 'precio_empresa_sinout']]


print(resumen_empresa_sinout.dtypes)

Fecha                    datetime64[ns]
Producto                         object
Mes                             float64
Año                             float64
precio_empresa_sinout           float64
dtype: object


In [6]:
#
resumen_empresa_sinout['Mes'] = resumen_empresa_sinout['Mes'].astype(int)
resumen_empresa_sinout['Año'] = resumen_empresa_sinout['Año'].astype(int)

In [7]:
resumen_empresa_sinout["posicion"] = pd.to_datetime(resumen_empresa_sinout["Año"].astype(str) + '-' + resumen_empresa_sinout["Mes"].astype(str) + '-01')
resumen_empresa_sinout['posicion'] = pd.to_datetime(resumen_empresa_sinout['posicion'])
resumen_empresa_sinout['time_delta_months'] = resumen_empresa_sinout['posicion'].dt.to_period('M').view(dtype='int64') - resumen_empresa_sinout ['Fecha'].dt.to_period('M').view(dtype='int64')

resumen_empresa_sinout['Embarque'] ='t+' + resumen_empresa_sinout['time_delta_months'].astype(str)

BROKER SIN OUTLIER

In [8]:
broker_sinout = base_sinout[base_sinout['Categoria'] == 'broker']
#Calculo los promedios 
resumen = broker_sinout.groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_comprador'].mean().reset_index()
resumen_vendedor = broker_sinout.groupby(['Fecha', 'Producto', 'Año', 'Mes'])['precio_vendedor'].mean().reset_index()

#Junto los promedios y redondeo a 2 decimales
resumen_broker_sinout = resumen.merge(resumen_vendedor, on=['Fecha', 'Producto', 'Año', 'Mes'], how='left')
resumen_broker_sinout['precio_comprador'] = resumen_broker_sinout['precio_comprador'].round(2)
resumen_broker_sinout['precio_vendedor'] = resumen_broker_sinout['precio_vendedor'].round(2)

# Calculo el precio promedio de la empresa
resumen_broker_sinout['precio_broker_sinout'] = round(resumen_broker_sinout[['precio_comprador', 'precio_vendedor']].mean(axis=1, skipna=True), 2)
#resumen_broker_sinout['precio_broker_sinout'] = round((resumen_broker_sinout['precio_comprador'] + resumen_broker_sinout['precio_vendedor']) / 2, 2)

# Elimino variables 
del resumen
del resumen_vendedor

resumen_broker_sinout  = resumen_broker_sinout[['Fecha', 'Producto', 'Mes', 'Año', 'precio_broker_sinout']]


In [9]:
resumen_broker_sinout['Mes'] = resumen_broker_sinout['Mes'].astype(int)
resumen_broker_sinout['Año'] = resumen_broker_sinout['Año'].astype(int)

In [10]:
resumen_broker_sinout["posicion"] = pd.to_datetime(resumen_broker_sinout["Año"].astype(str) + '-' + resumen_broker_sinout["Mes"].astype(str) + '-01')
resumen_broker_sinout ['posicion'] = pd.to_datetime(resumen_broker_sinout ['posicion'])

resumen_broker_sinout['time_delta_months'] = resumen_broker_sinout['posicion'].dt.to_period('M').view(dtype='int64') - resumen_broker_sinout['Fecha'].dt.to_period('M').view(dtype='int64')

resumen_broker_sinout['Embarque'] ='t+' + resumen_broker_sinout['time_delta_months'].astype(str)

TODO SIN OUTLIERS

In [14]:
full_sinout = bolsa.merge(resumen_broker_sinout, on=['Fecha', 'Producto', 'Mes', 'Año', 'posicion', 'time_delta_months', 'Embarque'], how='left')
full_sinout = full_sinout.merge(resumen_empresa_sinout, on=['Fecha', 'Producto', 'Mes', 'Año', 'posicion', 'time_delta_months', 'Embarque'], how='left')

print(full_sinout.dtypes)

Fecha                    datetime64[ns]
Producto                         object
Mes                               int64
Año                               int64
FOB_Bolsa                       float64
posicion                 datetime64[ns]
time_delta_months                 int64
Embarque                         object
precio_broker_sinout            float64
precio_empresa_sinout           float64
dtype: object


In [15]:
full_sinout = full_sinout.melt(id_vars=['Fecha', 'Producto', 'Embarque'], value_vars=[ 'FOB_Bolsa', 'precio_empresa_sinout', 'precio_broker_sinout'], var_name='Fuente', value_name='Precio')
full_sinout = full_sinout[full_sinout['Fecha'] > '2024-01-01']

In [16]:
resultado2 = full_sinout[full_sinout['Embarque'].isin(['t+0', 't+1', 't+2', 't+3'])]

for producto in resultado2['Producto'].unique():
    resultado_producto2 = resultado2[resultado2['Producto'] == producto]
    fig2 = px.line(resultado_producto2, x='Fecha', y='Precio', color='Fuente', facet_col='Embarque',
                  title='Precios ' + producto,
                  labels={'Precio': 'Precio', 'Fecha': 'Fecha', 'Fuente': 'Fuente'},
                  markers=True)  # Agrega marcadores

    fig2.update_traces(marker=dict(size=3))  # Ajusta el tamaño de los marcadores a 2
    
    fig2.show()
    fig2.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/precios_sinout" + producto + ".html")

c:\Users\evasquez\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


c:\Users\evasquez\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\evasquez\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\evasquez\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\evasquez\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\evasquez\AppData\Local\Programs\Python\Python312\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [17]:
# de la tabla full_sinout quiero hallar la diferencia entre los valores de la Fuente precio_empresa_sinout y precio_broker_sinout agrupado por fecha, producto y embarque
differences = full_sinout.pivot_table(values='Precio', index=['Fecha', 'Producto', 'Embarque'], columns='Fuente')
differences['Diferencia'] = differences['precio_broker_sinout'] - differences['precio_empresa_sinout']  #### BROKER - Empresa
differences.reset_index(inplace=True)
differences = differences[['Fecha', 'Producto', 'Embarque', 'Diferencia']]

differences = differences[differences['Embarque'].isin(['t+0', 't+1', 't+2', 't+3'])]
differences['Diferencia'] = differences['Diferencia'].round(2)

In [18]:
trigo = differences[differences['Producto'] == 'Trigo']
maiz = differences[differences['Producto'] == 'Maiz']
soja = differences[differences['Producto'] == 'Soja']
acgirasol = differences[differences['Producto'] == 'Aceite de Girasol']

h_soja = differences[differences['Producto'] == 'Harina de Soja']
ac_soja = differences[differences['Producto'] == 'Aceite de Soja']
maiz = differences[differences['Producto'] == 'Maiz']

TRIGO

In [19]:
# trigo
trigo['Fecha'] = pd.to_datetime(trigo['Fecha']).dt.strftime('%Y-%m-%d')

trigo_avg_diferencia = trigo.groupby('Embarque')['Diferencia'].mean().reset_index()
trigo = trigo.merge(trigo_avg_diferencia, on='Embarque', how='left').round(2)
trigo = trigo.dropna(subset=['Diferencia_x'])


C:\Users\evasquez\AppData\Local\Temp\ipykernel_27636\3480280546.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
import plotly.graph_objects as go

for embarque in trigo['Embarque'].unique():
    # Filtrar el DataFrame solo para el Embarque actual
    filtered_df = trigo[trigo['Embarque'] == embarque]
    
    # Crear un gráfico de barras para 'Diferencia_x' y agregar una línea para 'Diferencia_y'
    fig = px.bar(filtered_df, 
                 x='Fecha', 
                 y='Diferencia_x', 
                 labels={
                     "Fecha": "Fecha (Categoría)",
                     "Diferencia_x": "Diferencia X",
                 },
                 title=f"Gráfico de Diferencias para Embarque {embarque}",
                 opacity=0.9  # Mayor opacidad para las barras
                )

    # Cambiar el tipo de eje de la fecha a categoría
    fig.update_xaxes(type='category')

    # Cambiar el color de las barras a azul oscuro y ajustar el grosor de las barras
    fig.update_traces(marker_color='darkblue', marker_line_width=2, marker_line_color='darkblue')
    fig.update_layout(bargap=0.1)  # Reducir el espacio entre barras para hacerlas más gruesas

    # Asegurarnos de que las líneas de 'Diferencia_y' sean visibles
    fig.add_trace(
        go.Scatter(x=filtered_df['Fecha'], 
                   y=filtered_df['Diferencia_y'], 
                   mode='lines+markers', 
                   name=f'Diferencia_y ({embarque})',
                   legendgroup=embarque,
                   line=dict(width=2),  # Aumentar el grosor de las líneas
                   marker=dict(size=6))  # Aumentar el tamaño de los puntos en las líneas
    )

    # Ajustar el diseño del gráfico
    fig.update_layout(
        title=f"Gráfico de Diferencias para Embarque {embarque} (Barras y Líneas)",
        xaxis_title="Fecha",
        yaxis_title="Diferencia (Brokers - Empresas)",
        showlegend=True,
        height=600,  # Ajustar la altura del gráfico para más visibilidad
        width=1000  # Ajustar la anchura del gráfico
    )

    # Mostrar el gráfico
    fig.show()
    fig.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/trigo" + embarque + ".html")

ACEITE DE GIRASOL

In [21]:
acgirasol['Fecha'] = pd.to_datetime(acgirasol['Fecha']).dt.strftime('%Y-%m-%d')
acgirasol = acgirasol[~acgirasol['Fecha'].isin(['2024-01-26', '2024-04-23'])]
acgirasol_avg_diferencia = acgirasol.groupby('Embarque')['Diferencia'].mean().reset_index()
acgirasol = acgirasol.merge(acgirasol_avg_diferencia, on='Embarque', how='left').round(2)
acgirasol = acgirasol.dropna(subset=['Diferencia_x'])

C:\Users\evasquez\AppData\Local\Temp\ipykernel_27636\3080570217.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
for embarque in acgirasol['Embarque'].unique():
    # Filtrar el DataFrame solo para el Embarque actual
    filtered_df = acgirasol[acgirasol['Embarque'] == embarque]
    
    # Crear un gráfico de barras para 'Diferencia_x' y agregar una línea para 'Diferencia_y'
    fig2 = px.bar(filtered_df, 
                 x='Fecha', 
                 y='Diferencia_x', 
                 labels={
                     "Fecha": "Fecha (Categoría)",
                     "Diferencia_x": "Diferencia X",
                 },
                 title=f"Gráfico de Diferencias para Embarque {embarque}",
                 opacity=0.9  # Mayor opacidad para las barras
                )

    # Cambiar el tipo de eje de la fecha a categoría
    fig2.update_xaxes(type='category')

    # Cambiar el color de las barras a azul oscuro y ajustar el grosor de las barras
    fig2.update_traces(marker_color='darkblue', marker_line_width=2, marker_line_color='darkblue')
    fig2.update_layout(bargap=0.1)  # Reducir el espacio entre barras para hacerlas más gruesas

    # Asegurarnos de que las líneas de 'Diferencia_y' sean visibles
    fig2.add_trace(
        go.Scatter(x=filtered_df['Fecha'], 
                   y=filtered_df['Diferencia_y'], 
                   mode='lines+markers', 
                   name=f'Diferencia_y ({embarque})',
                   legendgroup=embarque,
                   line=dict(width=2),  # Aumentar el grosor de las líneas
                   marker=dict(size=6))  # Aumentar el tamaño de los puntos en las líneas
    )

    # Ajustar el diseño del gráfico
    fig2.update_layout(
        title=f"Gráfico de Diferencias para Embarque {embarque} (Barras y Líneas)",
        xaxis_title="Fecha",
        yaxis_title="Diferencia (Brokers - Empresas)",
        showlegend=True,
        height=600,  # Ajustar la altura del gráfico para más visibilidad
        width=1000  # Ajustar la anchura del gráfico
    )

    # Mostrar el gráfico
    fig2.show()
    fig2.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/acgirasol" + embarque + ".html")



SOJA

In [23]:
soja['Fecha'] = pd.to_datetime(soja['Fecha']).dt.strftime('%Y-%m-%d')
soja_avg_diferencia = soja.groupby('Embarque')['Diferencia'].mean().reset_index()
soja = soja.merge(soja_avg_diferencia, on='Embarque', how='left').round(2)
soja = soja.dropna(subset=['Diferencia_x'])

C:\Users\evasquez\AppData\Local\Temp\ipykernel_27636\1233868119.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
for embarque in soja['Embarque'].unique():
    # Filtrar el DataFrame solo para el Embarque actual
    filtered_df = soja[soja['Embarque'] == embarque]
    
    # Crear un gráfico de barras para 'Diferencia_x' y agregar una línea para 'Diferencia_y'
    fig3 = px.bar(filtered_df, 
                 x='Fecha', 
                 y='Diferencia_x', 
                 labels={
                     "Fecha": "Fecha (Categoría)",
                     "Diferencia_x": "Diferencia X",
                 },
                 title=f"Gráfico de Diferencias para Embarque {embarque}",
                 opacity=0.9  # Mayor opacidad para las barras
                )

    # Cambiar el tipo de eje de la fecha a categoría
    fig3.update_xaxes(type='category')

    # Cambiar el color de las barras a azul oscuro y ajustar el grosor de las barras
    fig3.update_traces(marker_color='darkblue', marker_line_width=2, marker_line_color='darkblue')
    fig3.update_layout(bargap=0.1)  # Reducir el espacio entre barras para hacerlas más gruesas

    # Asegurarnos de que las líneas de 'Diferencia_y' sean visibles
    fig3.add_trace(
        go.Scatter(x=filtered_df['Fecha'], 
                   y=filtered_df['Diferencia_y'], 
                   mode='lines+markers', 
                   name=f'Diferencia_y ({embarque})',
                   legendgroup=embarque,
                   line=dict(width=2),  # Aumentar el grosor de las líneas
                   marker=dict(size=6))  # Aumentar el tamaño de los puntos en las líneas
    )

    # Ajustar el diseño del gráfico
    fig3.update_layout(
        title=f"Gráfico de Diferencias para Embarque {embarque} (Barras y Líneas)",
        xaxis_title="Fecha",
        yaxis_title="Diferencia (Brokers - Empresas)",
        showlegend=True,
        height=600,  # Ajustar la altura del gráfico para más visibilidad
        width=1000  # Ajustar la anchura del gráfico
    )

    # Mostrar el gráfico
    fig3.show()
    fig3.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/soja" + embarque + ".html")


MAIZ

In [25]:
maiz['Fecha'] = pd.to_datetime(maiz['Fecha']).dt.strftime('%Y-%m-%d')
maiz_avg_diferencia = maiz.groupby('Embarque')['Diferencia'].mean().reset_index()
maiz = maiz.merge(maiz_avg_diferencia, on='Embarque', how='left').round(2)
maiz = maiz.dropna(subset=['Diferencia_x'])

C:\Users\evasquez\AppData\Local\Temp\ipykernel_27636\3816870272.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
for embarque in maiz['Embarque'].unique():
    # Filtrar el DataFrame solo para el Embarque actual
    filtered_df = maiz[maiz['Embarque'] == embarque]
    
    # Crear un gráfico de barras para 'Diferencia_x' y agregar una línea para 'Diferencia_y'
    fig4 = px.bar(filtered_df, 
                 x='Fecha', 
                 y='Diferencia_x', 
                 labels={
                     "Fecha": "Fecha (Categoría)",
                     "Diferencia_x": "Diferencia X",
                 },
                 title=f"Gráfico de Diferencias para Embarque {embarque}",
                 opacity=0.9  # Mayor opacidad para las barras
                )

    # Cambiar el tipo de eje de la fecha a categoría
    fig4.update_xaxes(type='category')

    # Cambiar el color de las barras a azul oscuro y ajustar el grosor de las barras
    fig4.update_traces(marker_color='darkblue', marker_line_width=2, marker_line_color='darkblue')
    fig4.update_layout(bargap=0.1)  # Reducir el espacio entre barras para hacerlas más gruesas

    # Asegurarnos de que las líneas de 'Diferencia_y' sean visibles
    fig4.add_trace(
        go.Scatter(x=filtered_df['Fecha'], 
                   y=filtered_df['Diferencia_y'], 
                   mode='lines+markers', 
                   name=f'Diferencia_y ({embarque})',
                   legendgroup=embarque,
                   line=dict(width=2),  # Aumentar el grosor de las líneas
                   marker=dict(size=6))  # Aumentar el tamaño de los puntos en las líneas
    )

    # Ajustar el diseño del gráfico
    fig4.update_layout(
        title=f"Gráfico de Diferencias para Embarque {embarque} (Barras y Líneas)",
        xaxis_title="Fecha",
        yaxis_title="Diferencia (Brokers - Empresas)",
        showlegend=True,
        height=600,  # Ajustar la altura del gráfico para más visibilidad
        width=1000  # Ajustar la anchura del gráfico
    )

    # Mostrar el gráfico
    fig4.show()
    fig4.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/maiz" + embarque + ".html")

HARINA DE SOJA

In [27]:
h_soja['Fecha'] = pd.to_datetime(h_soja['Fecha']).dt.strftime('%Y-%m-%d')
h_soja_avg_diferencia = h_soja.groupby('Embarque')['Diferencia'].mean().reset_index()
h_soja = h_soja.merge(h_soja_avg_diferencia, on='Embarque', how='left').round(2)
h_soja = h_soja.dropna(subset=['Diferencia_x'])

h_soja = h_soja[~h_soja['Fecha'].isin(['2024-07-16', '2024-07-31'])]

C:\Users\evasquez\AppData\Local\Temp\ipykernel_27636\2340927317.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
for embarque in h_soja['Embarque'].unique():
    # Filtrar el DataFrame solo para el Embarque actual
    filtered_df = h_soja[h_soja['Embarque'] == embarque]
    
    # Crear un gráfico de barras para 'Diferencia_x' y agregar una línea para 'Diferencia_y'
    fig5 = px.bar(filtered_df, 
                 x='Fecha', 
                 y='Diferencia_x', 
                 labels={
                     "Fecha": "Fecha (Categoría)",
                     "Diferencia_x": "Diferencia X",
                 },
                 title=f"Gráfico de Diferencias para Embarque {embarque}",
                 opacity=0.9  # Mayor opacidad para las barras
                )

    # Cambiar el tipo de eje de la fecha a categoría
    fig5.update_xaxes(type='category')

    # Cambiar el color de las barras a azul oscuro y ajustar el grosor de las barras
    fig5.update_traces(marker_color='darkblue', marker_line_width=2, marker_line_color='darkblue')
    fig5.update_layout(bargap=0.1)  # Reducir el espacio entre barras para hacerlas más gruesas

    # Asegurarnos de que las líneas de 'Diferencia_y' sean visibles
    fig5.add_trace(
        go.Scatter(x=filtered_df['Fecha'], 
                   y=filtered_df['Diferencia_y'], 
                   mode='lines+markers', 
                   name=f'Diferencia_y ({embarque})',
                   legendgroup=embarque,
                   line=dict(width=2),  # Aumentar el grosor de las líneas
                   marker=dict(size=6))  # Aumentar el tamaño de los puntos en las líneas
    )

    # Ajustar el diseño del gráfico
    fig5.update_layout(
        title=f"Gráfico de Diferencias para Embarque {embarque} (Barras y Líneas)",
        xaxis_title="Fecha",
        yaxis_title="Diferencia (Brokers - Empresas)",
        showlegend=True,
        height=600,  # Ajustar la altura del gráfico para más visibilidad
        width=1000  # Ajustar la anchura del gráfico
    )

    # Mostrar el gráfico
    fig5.show()
    fig5.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/h_soja" + embarque + ".html")

ACEITE DE SOJA

In [29]:
ac_soja['Fecha'] = pd.to_datetime(ac_soja['Fecha']).dt.strftime('%Y-%m-%d')
ac_soja_avg_diferencia = ac_soja.groupby('Embarque')['Diferencia'].mean().reset_index()
ac_soja = ac_soja.merge(ac_soja_avg_diferencia, on='Embarque', how='left').round(2)
ac_soja = ac_soja.dropna(subset=['Diferencia_x'])

C:\Users\evasquez\AppData\Local\Temp\ipykernel_27636\246371525.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
for embarque in ac_soja['Embarque'].unique():
    # Filtrar el DataFrame solo para el Embarque actual
    filtered_df = ac_soja[ac_soja['Embarque'] == embarque]
    
    # Crear un gráfico de barras para 'Diferencia_x' y agregar una línea para 'Diferencia_y'
    fig6 = px.bar(filtered_df, 
                 x='Fecha', 
                 y='Diferencia_x', 
                 labels={
                     "Fecha": "Fecha (Categoría)",
                     "Diferencia_x": "Diferencia X",
                 },
                 title=f"Gráfico de Diferencias para Embarque {embarque}",
                 opacity=0.9  # Mayor opacidad para las barras
                )

    # Cambiar el tipo de eje de la fecha a categoría
    fig6.update_xaxes(type='category')

    # Cambiar el color de las barras a azul oscuro y ajustar el grosor de las barras
    fig6.update_traces(marker_color='darkblue', marker_line_width=2, marker_line_color='darkblue')
    fig6.update_layout(bargap=0.1)  # Reducir el espacio entre barras para hacerlas más gruesas

    # Asegurarnos de que las líneas de 'Diferencia_y' sean visibles
    fig6.add_trace(
        go.Scatter(x=filtered_df['Fecha'], 
                   y=filtered_df['Diferencia_y'], 
                   mode='lines+markers', 
                   name=f'Diferencia_y ({embarque})',
                   legendgroup=embarque,
                   line=dict(width=2),  # Aumentar el grosor de las líneas
                   marker=dict(size=6))  # Aumentar el tamaño de los puntos en las líneas
    )

    # Ajustar el diseño del gráfico
    fig6.update_layout(
        title=f"Gráfico de Diferencias para Embarque {embarque} (Barras y Líneas)",
        xaxis_title="Fecha",
        yaxis_title="Diferencia (Brokers - Empresas)",
        showlegend=True,
        height=600,  # Ajustar la altura del gráfico para más visibilidad
        width=1000  # Ajustar la anchura del gráfico
    )

    # Mostrar el gráfico
    fig6.show()
    fig6.write_html("C:/Users/evasquez/OneDrive - Bolsa de Cereales de Buenos Aires/Documentos - Instituto Estudios Económicos/Pedidos/2024/Diferencias_FOB/Reunion_Ministerio_1508/análisis/graficos/ac_soja" + embarque + ".html")